In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce


In [2]:
df = pd.read_csv('/kaggle/input/spaceshipt-df/train.csv')


In [3]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [4]:
df = df[['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported']].copy()\
.reset_index(drop=True)

In [5]:
df.groupby('HomePlanet').size()
# Replace Na Values with the mode of the planets
planets_mode = df['HomePlanet'].mode(dropna=True)[0]
df['HomePlanet'] = df['HomePlanet'].fillna(planets_mode).copy()




In [6]:
cryosleep_mode = df['CryoSleep'].mode(dropna=True)[0]
df['CryoSleep'] = df['CryoSleep'].fillna(cryosleep_mode).copy()


/tmp/ipykernel_17/3772733588.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['CryoSleep'] = df['CryoSleep'].fillna(cryosleep_mode).copy()


In [7]:
# Replacing na values from destination with mode of it!
destination_mode = df['Destination'].mode(dropna=True)[0]
df['Destination'] = df['Destination'].fillna(destination_mode).copy()



In [8]:
# Replacing Na values from age by it`s mean
age_mean = round(df['Age'].mean(skipna=True))
df['Age'] = df['Age'].fillna(age_mean).copy()

In [9]:
#Removing VIP , Cabin and NAME rows witha na values
df = df.dropna(subset=['VIP','Cabin','Name']).copy().reset_index(drop=True)


In [10]:
df['Transported'] = df['Transported'].astype(int)
df['CryoSleep'] = df['CryoSleep'].astype(int)

In [11]:
X = df.drop(['Transported'],axis = 1)
y = df['Transported']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state =3000)

In [13]:
encoder = ce.OrdinalEncoder(cols=['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)


RandomForestClassifier()

In [14]:
y_pred = model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score

print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 100 decision-trees : 0.7178


In [16]:
test = pd.read_csv('/kaggle/input/spaceshiptest/test.csv')
test_predict = encoder.fit_transform(test )

In [17]:
prediction = model.predict(test_predict)
prediction = prediction.astype(bool)


In [18]:
gender_submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Transported':prediction})

In [19]:
gender_submission.to_csv('./gender_submission_new_space_ship.csv',index=False)  